In [ ]:
%pip install -qU langchain langchain-community langchain-openai python-dotenv numexpr

In [16]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains.llm_math.base import LLMMathChain
from langchain_community.utilities import BingSearchAPIWrapper
from langchain.agents import Tool, initialize_agent, tool

In [12]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
llm = ChatOpenAI(
    model = model_name,
    openai_api_key = openai_api_key,
    temperature = 1,
    max_tokens = 1000
)

In [8]:
llm_math = LLMMathChain.from_llm(llm)
math_tool = Tool(
    name = "Calculator",
    description = "Useful tool when you need to answer questions related to simple mathematics",
    func = llm_math.run
)

In [9]:
@tool("BingSearch")
def search(query_string: str):
    """
    Useful to search for any kind of information and
    when you need to search the internet for any kinds of detailed information.
    """

    search = BingSearchAPIWrapper()
    return search.run(query_string)

In [ ]:
tools = [math_tool, search]

agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 10,
    handle_parsing_errors = True
)

In [ ]:
response = agent("let me know the capital of Malaysia and whats the root over 25?")

print(response)